[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klavis-ai/klavis/blob/main/examples/claude/Use_Klavis_with_Claude.ipynb)


# Claude + Klavis AI Integration

This tutorial demonstrates how to use Anthropic's Claude with tool use (function calling) with Klavis MCP (Model Context Protocol) servers.


## Prerequisites

- **Anthropic API key** - Get at [console.anthropic.com](https://console.anthropic.com/)
- **Klavis API key** - Get at [klavis.ai](https://klavis.ai/)


In [1]:
# Install the required packages
%pip install -qU anthropic klavis

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import json
from anthropic import Anthropic
from klavis import Klavis
from klavis.types import McpServerName, ConnectionType, ToolFormat

# Set environment variables
os.environ["ANTHROPIC_API_KEY"] = "YOUR_ANTHROPIC_API_KEY"  # Replace with your actual Anthropic API key
os.environ["KLAVIS_API_KEY"] = "YOUR_KLAVIS_API_KEY"  # Replace with your actual Klavis API key


## Case Study 1 : Claude + YouTube MCP Server


#### Step 1 - Create YouTube MCP Server using Klavis


In [4]:
klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

youtube_mcp_instance = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.YOUTUBE,
    user_id="1234",
    platform_name="Klavis"
)

# print(f"🔗 YouTube MCP server created at: {youtube_mcp_instance.server_url}, and the instance id is {youtube_mcp_instance.instance_id}")


#### Step 2 - Create general method to use MCP Server with Claude


In [10]:
def claude_with_mcp_server(mcp_server_url: str, user_query: str):
    claude_client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

    messages = [
        {"role": "user", "content": f"{user_query}"}
    ]
    
    mcp_server_tools = klavis_client.mcp_server.list_tools(
        server_url=mcp_server_url,
        format=ToolFormat.ANTHROPIC,
    )
    
    response = claude_client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=4000,
        system="You are a helpful assistant. Use the available tools to answer the user's question.",
        messages=messages,
        tools=mcp_server_tools.tools
    )
    
    messages.append({"role": "assistant", "content": response.content})

    if response.stop_reason == "tool_use":
        tool_results = []
        
        for content_block in response.content:
            if content_block.type == "tool_use":
                function_name = content_block.name
                function_args = content_block.input
                
                print(f"🔧 Calling: {function_name}, with args: {function_args}")
                
                result = klavis_client.mcp_server.call_tools(
                    server_url=mcp_server_url,
                    tool_name=function_name,
                    tool_args=function_args,
                    connection_type=ConnectionType.STREAMABLE_HTTP
                )
                
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": content_block.id,
                    "content": str(result)
                })
        
        messages.append({"role": "user", "content": tool_results})
            
        final_response = claude_client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=4000,
            system="You are a helpful assistant. Use the available tools to answer the user's question.",
            messages=messages,
            tools=mcp_server_tools.tools
        )
        
        return final_response.content[0].text
    else:
        return response.content[0].text


#### Step 3 - Summarize your favorite video!


In [11]:
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=LCEmiRjPEtQ"  # pick a video you like!

result = claude_with_mcp_server(
    mcp_server_url=youtube_mcp_instance.server_url, 
    user_query=f"Please provide a complete summary of this YouTube video with timestamp: {YOUTUBE_VIDEO_URL}"
)

print(result)


🔧 Calling: get_youtube_video_transcript, with args: {'url': 'https://www.youtube.com/watch?v=LCEmiRjPEtQ'}
Based on the video details, I'll provide a comprehensive summary of Andrej Karpathy's keynote speech "Software Is Changing (Again)" at AI Startup School. Here's a breakdown by key segments:

1. Introduction and Software Evolution (0:00-04:40)
- The talk begins with an overview of how software is fundamentally changing
- Discusses the evolution from Software 1.0 to 3.0
- Karpathy positions this as a major paradigm shift in software development

2. Programming in English & Software 3.0 (04:40-06:10)
- Introduces the concept of programming in natural language
- Explains how LLMs are changing the programming paradigm
- Highlights the transition to Software 3.0 where English becomes the programming interface

3. LLMs as Computing Infrastructure (06:10-14:39)
- Discusses three key aspects of LLMs:
  - As utilities
  - As fabrication facilities (fabs)
  - As operating systems
- Draws his

✅ Great! You've successfully created an AI agent that uses Claude's tool use with Klavis MCP servers to summarize YouTube videos!


## Case Study 2 : Claude + Gmail MCP Server (OAuth needed)


In [12]:
import webbrowser

gmail_mcp_server = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.GMAIL,
    user_id="1234",
    platform_name="Klavis",
    connection_type=ConnectionType.STREAMABLE_HTTP,
)

webbrowser.open(gmail_mcp_server.oauth_url)

print(f"🔐 Opening OAuth authorization for Gmail, if you are not redirected, please open the following URL in your browser: {gmail_mcp_server.oauth_url}")

🔐 Opening OAuth authorization for Gmail, if you are not redirected, please open the following URL in your browser: https://api.klavis.ai/oauth/gmail/authorize?instance_id=d9d482b3-433a-4330-9a8b-9548c0b0a326


In [13]:
EMAIL_RECIPIENT = "zihaolin@klavis.ai" # Replace with your email
EMAIL_SUBJECT = "Test Claude + Gmail MCP Server"
EMAIL_BODY = "Hello World from Claude!"

result = claude_with_mcp_server(
    mcp_server_url=gmail_mcp_server.server_url, 
    user_query=f"Please send an email to {EMAIL_RECIPIENT} with subject {EMAIL_SUBJECT} and body {EMAIL_BODY}"
)

print(result)

🔧 Calling: send_email, with args: {'to': ['zihaolin@klavis.ai'], 'subject': 'Test Claude + Gmail MCP Server', 'body': 'Hello World from Claude!'}
I've sent the email as requested with:
- To: zihaolin@klavis.ai
- Subject: Test Claude + Gmail MCP Server
- Body: Hello World from Claude!

The email was sent successfully. Is there anything else you'd like me to help you with?


## Summary

This tutorial demonstrated how to integrate Anthropic's Claude with tool use capabilities with Klavis MCP servers to create powerful AI applications. We covered two practical examples:

**🎥 YouTube Integration**: Built an AI assistant that can automatically summarize YouTube videos by extracting transcripts and providing detailed, timestamped summaries.

**📧 Gmail Integration**: Created an AI-powered email assistant that can send emails through Gmail with OAuth authentication.

### Key Takeaways:
- **Easy Setup**: Klavis MCP servers can be created with just a few lines of code
- **Claude Compatible**: All tools are formatted for seamless Claude tool use
- **Versatile**: Support for both simple APIs (YouTube) and OAuth-authenticated services (Gmail)
- **Scalable**: The same pattern can be applied to any of the MCP servers available in Klavis
- **Advanced Reasoning**: Claude's superior reasoning capabilities make it excellent for complex analysis tasks

**Happy building!** 🚀
